In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/movies_overview.csv')
df.head()

,title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,12 Angry Men,The defense and the prosecution have rested an...,[18]


# EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9980 entries, 0 to 9979
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      9980 non-null   object
 1   overview   9980 non-null   object
 2   genre_ids  9980 non-null   object
dtypes: object(3)
memory usage: 234.0+ KB


In [4]:
df["genre_ids"].value_counts()

genre_ids
[18]                           577
[35]                           571
[18, 10749]                    273
[35, 10749]                    246
[35, 18]                       232
                              ... 
[28, 53, 18, 12]                 1
[16, 35, 878, 10770, 10749]      1
[878, 53, 80, 9648]              1
[16, 878, 10751, 12]             1
[18, 36, 35, 10749]              1
Name: count, Length: 2222, dtype: int64

In [5]:
from ast import literal_eval
from itertools import chain
from collections import Counter

df_genres = df.copy()

genre_list_id = list(chain.from_iterable(df["genre_ids"].apply(literal_eval)))

id_name_map = pd.read_csv('data/movies_genres.csv').to_dict(orient='records')
id_name_map = {item['id']: item['name'] for item in id_name_map}

genre_list = [id_name_map[genre_id] for genre_id in genre_list_id if genre_id in id_name_map]

genre_counter = Counter(genre_list)

genre_counter

Counter({'Drama': 4523,
         'Comedy': 3626,
         'Thriller': 2757,
         'Action': 2349,
         'Adventure': 1700,
         'Romance': 1699,
         'Crime': 1573,
         'Horror': 1475,
         'Science Fiction': 1235,
         'Fantasy': 1154,
         'Family': 1134,
         'Mystery': 966,
         'Animation': 910,
         'History': 490,
         'War': 324,
         'Music': 279,
         'Western': 152,
         'TV Movie': 119})

In [6]:
from pprint import pprint

pprint(id_name_map)
pprint(df[["overview", "genre_ids"]].sample(5).to_dict(orient='records'))

{12: 'Adventure',
 14: 'Fantasy',
 16: 'Animation',
 18: 'Drama',
 27: 'Horror',
 28: 'Action',
 35: 'Comedy',
 36: 'History',
 37: 'Western',
 53: 'Thriller',
 80: 'Crime',
 99: 'Documentary',
 878: 'Science Fiction',
 9648: 'Mystery',
 10402: 'Music',
 10749: 'Romance',
 10751: 'Family',
 10752: 'War',
 10770: 'TV Movie'}
[{'genre_ids': '[35, 10749]',
  'overview': 'At a tiny Parisian café, the adorable yet painfully shy Amélie '
              'accidentally discovers a gift for helping others. Soon Amelie '
              'is spending her days as a matchmaker, guardian angel, and '
              'all-around do-gooder. But when she bumps into a handsome '
              'stranger, will she find the courage to become the star of her '
              'very own love story?'},
 {'genre_ids': '[28, 10752]',
  'overview': 'In a kingdom ruled by a young and unpredictable king, the '
              'military commander has a secret weapon: a shadow, a look-alike '
              'who can fool both 

In [7]:
import re

def remove_extra_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)


df["overview"] = df["overview"].apply(remove_punctuation)
df["overview"] = df["overview"].apply(remove_extra_spaces)

In [8]:
import plotly.graph_objects as go

word_count = df["overview"].str.split().str.len()

fig = go.Figure(data=[go.Histogram(x=word_count)])

fig.update_layout(
    title="Distribution of Word Count in Movie Overviews",
    xaxis_title="Word Count",
    yaxis_title="Frequency",
    xaxis=dict(
        tickmode='linear',
        dtick=1,
        range=[0, 100]
    )
)
fig.show()

In [9]:
from collections import Counter

def most_frequent_words(texts, n=10):
    words = [word for txt in texts for word in txt.split()]

    word_counts = Counter(words)

    most_common = dict(word_counts.most_common(n))

    fig = go.Figure(
        go.Bar(
            x=list(most_common.keys()),
            y=list(most_common.values())
        )
    )
    fig.update_layout(
        title_text=f"Top {n} most frequent words in the text",
        title_x=0.5,  # Center the title
        xaxis_title="Words",
        yaxis_title="Frequency",
        xaxis_tickangle=-45,
    )
    fig.show()
most_frequent_words(df["overview"], 40)


# Preprocessing

In [10]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

list(stop_words)[:10]

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['no', 'by', 'for', "it's", 'once', 'those', 'do', 'now', 'shouldn', 'have']

In [13]:
!python -m spacy download en_core_web_sm

^C


In [12]:
import spacy

nlp = spacy.load("en_core_web_sm")

stop_words_spacy = nlp.Defaults.stop_words

list(stop_words_spacy)[:10]

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.